In [ ]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

In [ ]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

In [ ]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('simple_chatbot.h5')

In [ ]:
def clean_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [ ]:
def bag_of_words(sentence):
    sentence_words = clean_sentence(sentence)
    bag = [0] * len(words)
    
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
                
    return np.array(bag)

In [ ]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    err_tresh = 0.25
    
    result = [[i, r] for i, r in enumerate(res) if r > err_tresh]
    result.sort(key=lambda x: x[1], reverse=True)
    result_list = []
    for r in result:
        result_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
        
    return result_list

In [ ]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result, tag

In [ ]:
while True:
    message = input("")
    ints = predict_class(message)
    res, tag = get_response(ints, intents)
    print('tag:', tag, 'res:', res)
    
    if tag == 'goodbye':
        break